In [1]:
import pandas as pd
from text_preprocessing_funcs import *

In [2]:
df = pd.read_json('raw_data.json')
df.head()

,url,name,type,price,prefecture,smv,acidity,kw_word,kw_wines,kw_beer,kw_foods,description
0,https://www.truesake.com/collections/all/produ...,"Dassai 45 Junmai Daiginjo ""Otter Festival""",Junmai Daiginjo,30.00,Yamaguchi,+3,1.4,Chunky,Pinot Noir / Chewy Whites,Ales,"Mushroom risotto, tempura, fried chicken, cavi...",This sake has an older brother Daiginjo that h...
1,https://www.truesake.com/collections/all/produ...,"Kikusui Junmai Ginjo ""Chrysanthemum Water""",Junmai Ginjo,33.00,Niigata,+1,1.6,Citrus,Sharp Whites/Dry Reds,Ales/Ambers,Suitable for light and stronger flavored dishe...,True Sake owner Beau Timken recorded K...
2,https://www.truesake.com/collections/all/produ...,"Wakatake Onikoroshi Junmai Daiginjo ""Demon Sla...",Junmai Daiginjo,49.00,Shizuoka,+/-0,1.4,Popular,Elegant reds/Complex whites,Crisp Ales,Cuisines that like big white wines.,Talk about a nice aroma package on thi...
3,https://www.truesake.com/collections/all/produ...,"Shunnoten Tokubetsu Junmai ""Fishermen Cup""",Tokubetsu Junmai,9.00,Yamanashi,+3,1.6,Smooth,Pinot Noir/Slippery Whites,Gentle Ales,None,The nose on this awesome looking “catch cup” i...
4,https://www.truesake.com/collections/all/produ...,"Kenbishi Kuromatsu Honjozo ""Black Pine""",Honjozo,40.00,Hyogo,±0.5,1.5,Rich,Deep Reds/Fat Whites,Stouts,Perfect for pub food salty and savory and grea...,The nose on this extremely famous Honjozo from...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 322 entries, 0 to 321
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   url          322 non-null    object 
 1   name         322 non-null    object 
 2   type         303 non-null    object 
 3   price        322 non-null    object 
 4   prefecture   318 non-null    object 
 5   smv          304 non-null    object 
 6   acidity      277 non-null    float64
 7   kw_word      288 non-null    object 
 8   kw_wines     286 non-null    object 
 9   kw_beer      285 non-null    object 
 10  kw_foods     280 non-null    object 
 11  description  307 non-null    object 
dtypes: float64(1), object(11)
memory usage: 32.7+ KB


In [7]:
cols_to_fix_dict = {
    'kw_word': ['WINE'],
    'kw_wines': ['BEER'],
    'kw_beer': ['FOOD', 'BEER:', 'Foods:'],
    'kw_foods': ['Updated', '('],
    'description': ['WORD']
}

In [8]:
def clean_df_text_cols(df, cols_to_fix_dict):
    for col, text_list in cols_to_fix_dict.items():
        if col in df.columns.tolist():
            for text in text_list:   
                # Apply split_on_chars func to parse for the right text
                df[col] = df[col].apply(lambda d: split_on_chars(text, d))
            # Clean text
            df[col] = df[col].apply(lambda d: clean_text(d))   
    return df

In [9]:
df = clean_df_text_cols(df, cols_to_fix_dict)

In [10]:
df.head()

,url,name,type,price,prefecture,smv,acidity,kw_word,kw_wines,kw_beer,kw_foods,description
0,https://www.truesake.com/collections/all/produ...,"Dassai 45 Junmai Daiginjo ""Otter Festival""",Junmai Daiginjo,30.00,Yamaguchi,+3,1.4,chunky,pinot noir chewy whites,ales,mushroom risotto tempura fried chicken caviar ...,this sake has an older brother daiginjo that h...
1,https://www.truesake.com/collections/all/produ...,"Kikusui Junmai Ginjo ""Chrysanthemum Water""",Junmai Ginjo,33.00,Niigata,+1,1.6,citrus,sharp whites dry reds,ales ambers,suitable for light and stronger flavored dishe...,true sake owner beau timken recorded kikusui a...
2,https://www.truesake.com/collections/all/produ...,"Wakatake Onikoroshi Junmai Daiginjo ""Demon Sla...",Junmai Daiginjo,49.00,Shizuoka,+/-0,1.4,popular,elegant reds complex whites,crisp ales,cuisines that like big white wines,talk about a nice aroma package on this ultrap...
3,https://www.truesake.com/collections/all/produ...,"Shunnoten Tokubetsu Junmai ""Fishermen Cup""",Tokubetsu Junmai,9.00,Yamanashi,+3,1.6,smooth,pinot noir slippery whites,gentle ales,none,the nose on this awesome looking catch cup is ...
4,https://www.truesake.com/collections/all/produ...,"Kenbishi Kuromatsu Honjozo ""Black Pine""",Honjozo,40.00,Hyogo,±0.5,1.5,rich,deep reds fat whites,stouts,perfect for pub food salty and savory and grea...,the nose on this extremely famous honjozo from...
